In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 30.1 MB/s eta 0:00:00


In [8]:
 !pip install datasets sentencepiece optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.1 MB/s eta 0:00:00


In [9]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.1 MB/s eta 0:00:00


In [10]:
import json
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments,EarlyStoppingCallback
import torch
from transformers import Seq2SeqTrainer
from torch.utils.data import Dataset, DataLoader
from itertools import product
import optuna
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

nlp = spacy.load("en_core_web_sm")
# download necessary resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
import json
with open('/content/drive/MyDrive/MTP Project/t5_data/true_300_attr.json') as jsonfile:
    train_data = json.load(jsonfile)

In [5]:
train_data[0]
#from annotated 300 samples, prepare the input and output dataset
#for input , append sentence + What is the attribute related to entity + {entity value} + with quantity {quantity value}'
#for output , take the attribute value


{'input': 'As of 2016 , Solar cell efficiency exceeds 10 % .  What is the attribute related to entity Solar cell with quantity 10 %?',
 'output': 'efficiency'}

In [11]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is your list of dictionaries
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42, stratify=None)

# train_data contains the training samples
# val_data contains the validation samples

print(f"Total Training dataset samples : {len(train_data)}")
print(f"Total validation set samples : {len(val_data)}")


df_train = pd.DataFrame(train_data)
df_val=pd.DataFrame(val_data)

Total Training dataset samples : 270
Total validation set samples : 30


In [12]:
def preprocess(row,max_input_len,max_target_len,tok):
    tok=tok
    # Retrieve the input and output strings from the row dictionary
    input_str = row['input']
    target_str = row['output']
    # Tokenize the input string with a maximum length and truncation
    tokenized_inputs = tok(input_str, max_length=max_input_len, truncation=True)
    # Tokenize the target string with a maximum length and truncation
    tokenized_targets = tok(text_target=target_str, max_length=max_target_len, truncation=True)
    # Assign the tokenized target's 'input_ids' to the 'labels' key in the tokenized input dictionary
    tokenized_inputs['labels'] = tokenized_targets['input_ids']
    # Return the tokenized input dictionary
    return tokenized_inputs
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return a dictionary containing the input_ids, attention_mask, and labels for a given index
        return {
            'input_ids': self.data[idx]['input_ids'],
            'attention_mask': self.data[idx]['attention_mask'],
            'labels': self.data[idx]['labels'],

        }



In [13]:
#load model parameters with cv of k=3
with open("/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/final_params.json") as f:
  params=json.load(f)
print(params)

{'dropout_rate': 0.02154867810249722, 'input_seq_length': 256, 'learning_rate': 0.00022763326997820923, 'num_train_epochs': 24, 'output_seq_length': 14, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.0013437781293502878}


In [14]:
params={"dropout_rate": 0.02154867810249722, "input_seq_length": 256, "learning_rate": 0.00022763326997820923,
        "num_train_epochs": 24, "output_seq_length": 14, "per_device_eval_batch_size": 16, "per_device_train_batch_size": 16,
        "weight_decay": 0.0013437781293502878}
print(params)

{'dropout_rate': 0.02154867810249722, 'input_seq_length': 256, 'learning_rate': 0.00022763326997820923, 'num_train_epochs': 24, 'output_seq_length': 14, 'per_device_eval_batch_size': 16, 'per_device_train_batch_size': 16, 'weight_decay': 0.0013437781293502878}


In [15]:

# Specify the model name
model_name = "t5-base"
# Load the pre-trained T5 model for sequence-to-sequence learning
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,dropout_rate=params['dropout_rate'])
# Initialize the tokenizer for the T5 model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
# Create the training dataset by applying the preprocess function to each row in df_train
train_dataset = df_train.apply(lambda row: preprocess(row,params['input_seq_length'],params['output_seq_length'],tokenizer), axis=1)

# Create the validation dataset by applying the preprocess function to each row in df_val
val_dataset = df_val.apply(lambda row: preprocess(row, params['input_seq_length'],params['output_seq_length'],tokenizer), axis=1)

# Convert the train_dataset from a DataFrame to a list
train_dataset=train_dataset.to_list()
# Convert the val_dataset from a DataFrame to a list
val_dataset=val_dataset.to_list()
# Create an instance of the MyDataset class for the training dataset
train_dataset = MyDataset(train_dataset)

# Create an instance of the MyDataset class for the validation dataset
val_dataset = MyDataset(val_dataset)

In [17]:
#define training argumentrs
dir='/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/checkpoints/'
training_args = Seq2SeqTrainingArguments(

  output_dir=dir,
  evaluation_strategy="epoch",
  save_strategy="epoch",
   logging_strategy='epoch',
  learning_rate=params['learning_rate'],
  per_device_train_batch_size=params['per_device_train_batch_size'],
  per_device_eval_batch_size=params['per_device_eval_batch_size'],
  weight_decay=params['weight_decay'],
  save_total_limit=1,
  num_train_epochs=params['num_train_epochs'],
  predict_with_generate=True,
  load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False




)

In [19]:
#function defined to apply postprocessing to predicted and label text
def post_process(text):
  # Convert the text to lowercase
  text = text.lower()
  # Tokenize the text using NLTK's word_tokenize
  tokens = nltk.word_tokenize(text)
  # Loop through each token in the list
  for i in range(len(tokens)):
     w=tokens[i]
      # Remove any spaces in the token
     w=w.replace(" ","")

     # Replace the token in the list with the modified one
     tokens[i]=w
  # Get the set of English stopwords
  stop_words = set(stopwords.words('english'))
  # Filter out the stopwords from the tokens list
  filtered_tokens = [word for word in tokens if word not in stop_words]

  # Join the tokens back into a preprocessed text string
  preprocessed_text = ' '.join(tokens)

  # Apply lemmatization to the preprocessed text using spaCy
  doc = nlp(preprocessed_text)

  lemmatized_text = " ".join([token.lemma_ for token in doc])

  # Remove any non-alphanumeric characters from the lemmatized text using regular expression
  text = re.sub('[^a-zA-Z0-9\s]', '',  lemmatized_text)
  # Tokenize the text again to get the final list of tokens
  tokens = nltk.word_tokenize(text)
  return tokens

In [ ]:
def compute_metrics(eval_pred):
    # Initialize the tokenizer for the T5 model
    tokenizer = AutoTokenizer.from_pretrained("t5-base")

    # Unpack the evaluation predictions tuple into preds and labels
    preds, labels = eval_pred

    # Initialize lists for precision and recall
    prec, recall = [], []

    # If the predictions are in a tuple, extract the first element
    if isinstance(preds, tuple):
        preds = preds[0]

    # Decode the predicted and label sequences into text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Iterate through each pair of decoded predictions and labels
    for i in range(len(decoded_preds)):
        # Tokenize and post-process the predicted and label sequences
        pred_tokens = post_process(decoded_preds[i])
        label_tokens = post_process(decoded_labels[i])

        # Calculate the overlap between predicted and label tokens
        overlap = len(set(pred_tokens).intersection(set(label_tokens)))

        # Calculate relaxed precision
        if len(pred_tokens) != 0:
            rp = overlap / len(pred_tokens)
        else:
            rp = 0
        prec.append(rp)

        # Calculate relaxed recall
        if len(label_tokens) != 0:
            rr = overlap / len(label_tokens)
        else:
            rr = 0
        recall.append(rr)

    # Calculate precision, recall, and F1-score
    precision = np.mean(prec)
    recall = np.mean(recall)
    f1 = (2 * precision * recall) / (precision + recall)

    # Create a dictionary of the computed metrics
    res = {'f1': f1, 'Relaxed Precision': precision, 'Relaxed Recall': recall}

    return res


In [20]:
def compute_metrics(eval_pred):
  # Initialize the tokenizer for the T5 model
  tokenizer = AutoTokenizer.from_pretrained("t5-base")

  # Unpack the evaluation predictions tuple into preds and labels
  preds, labels = eval_pred

  # Initialize lists for precision and recall
  prec,recall=[],[]
  if isinstance(preds, tuple):
       preds = preds[0]
  # Decode the predicted and label sequences
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   # Replace -100 in the labels as we can't decode them.
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Iterate through each pair of decoded predictions and labels
  for i in range(len(decoded_preds)):
        # Tokenize and post-process the predicted and label sequences
        pred_tokens = post_process(decoded_preds[i])
        label_tokens = post_process(decoded_labels[i])

         # calculate overlap
        overlap = len(set(pred_tokens).intersection(set(label_tokens)))
        # calculate relaxed precision for one sample
        if(len(pred_tokens)!=0):
           rp = overlap / len(pred_tokens)
        else:
           rp=0
        prec.append(rp)
        # calculate relaxed recall for one sample
        if(len(label_tokens)!=0):

           rr = overlap / len(label_tokens)
        else:
             rr=0
        recall.append(rr)
  # Calculate precision, recall, and F1-score of all samples
  precision = np.mean(prec)
  recall =np.mean(recall)
  f1= (2*precision*recall)/(precision+recall)

  res={'f1':f1,'Relaxed Precision':precision,'Relaxed Recall': recall}

  return res



In [ ]:
#train the model
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,

    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics

)
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Relaxed precision,Relaxed recall
1,2.396800,1.171298,0.611098,0.613889,0.608333
2,0.678600,1.248522,0.580986,0.605556,0.558333
3,0.247600,1.303525,0.606407,0.588889,0.625000
4,0.079700,1.605120,0.620805,0.616667,0.625000
5,0.061700,1.806502,0.669260,0.658333,0.680556
6,0.040000,1.804771,0.665205,0.672222,0.658333
7,0.012200,1.689873,0.640202,0.633333,0.647222
8,0.004100,1.705456,0.595675,0.605556,0.586111
9,0.007600,1.732047,0.572397,0.600000,0.547222
10,0.002300,1.820612,0.605797,0.633333,0.580556


TrainOutput(global_step=408, training_loss=0.14744364844941005, metrics={'train_runtime': 589.7549, 'train_samples_per_second': 10.988, 'train_steps_per_second': 0.692, 'total_flos': 602011964620800.0, 'train_loss': 0.14744364844941005, 'epoch': 24.0})

In [ ]:
logs=trainer.state.log_history

In [ ]:
import json
with open('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/logs_new.json','w') as f:
  json.dump(logs,f)


In [ ]:
#save the pretrained model
model.save_pretrained('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_model')

In [ ]:
#save tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer')

('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer/spiece.model',
 '/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/t5_attr_tokenizer/tokenizer.json')

In [ ]:
train_loss,val_loss,prec,recall,f1={},{},{},{},{}
for i in logs:

  if 'loss' in i:
    train_loss[i['epoch']]=i['loss']

  elif 'eval_loss' in i:
    val_loss[i['epoch']]=i['eval_loss']
    prec[i['epoch']]= i['eval_Relaxed Precision']
    recall[i['epoch']]=i['eval_Relaxed Recall']
    f1[i['epoch']]=i['eval_f1']


In [ ]:
 #load the test data with 25 samples of each seq length
with open('/content/drive/MyDrive/MTP Project/t5_data/test_data_100_seperate.json' ) as f:
  rule_test=json.load(f)
len(rule_test)

100

In [ ]:
test_json=[]
for i in rule_test:
  d={}
  inp=i['sentence']+" "+ f"What is the attribute related to entity {i['entity']} with quantity {i['quantity']}?"

  outt=i['measuring attribute']
  d['input']=inp
  d['output']=outt
  d['entity']=i['entity']#rule_test[i][0]
  d['quantity']=i['quantity']#rule_test[i][2]
  d['sen']=i['sentence']
  test_json.append(d)
len(test_json)

100

In [ ]:
test_json[0]

{'input': 'The total length of this LPG pipeline is 1415 km . What is the attribute related to entity this LPG pipeline with quantity 1415 km?',
 'output': 'total length',
 'entity': 'this LPG pipeline',
 'quantity': '1415 km',
 'sen': 'The total length of this LPG pipeline is 1415 km .'}

In [ ]:
#predict the attribute given sentence,entity,quantity
#list of dictionaries with senetence and attribute as keys
res=[]
for i in range(len(test_json)):

  d=test_json[i]

  input=d['input']

  tokenized_inputs= tokenizer(input, max_length=256, truncation=True,return_tensors='pt')
  tokenized_inputs=tokenized_inputs.to('cuda')
  output_sequence = model.generate(**tokenized_inputs
                                 )

# decode the output sequence into text using the tokenizer
  #d=rule_test[i]
  u={}
  print(d['sen'])
  print("------------------------------Rule based result-------------------------------")
  print(f" Entity : {d['entity']}")
  print(f" Quantity : {d['quantity']} ")
  print(f" Attribute : {d['output']} ")
  print("-------------------------------- T5 Result------------------------------------")
  output_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
  print(f" Attribute : {output_text}")
  print('\n\n')
  u['sentence']=d['sen']
  u['attribute']=output_text
  res.append(u)

In [ ]:
#store the result for calculation of metrics in json file
import json
with open ('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/test_25samples_prediction.json','w') as f:
  json.dump(res,f)


In [ ]:
 #load the new test data according to distribution
 with open('/content/drive/MyDrive/MTP Project/t5_data/new_test_data_100.json' ) as f:
  rule_test=json.load(f)
len(rule_test)

100

In [ ]:
test_json=[]
for i in rule_test:
  d={}
  inp=i['sentence']+" "+ f"What is the attribute related to entity {i['entity']} with quantity {i['quantity']}?"


  outt=i['measuring attribute']
  d['input']=inp
  d['output']=outt
  d['entity']=rule_test[i][0]
  d['quantity']=rule_test[i][2]
  d['sen']=i
  test_json.append(d)
len(test_json)

100

In [ ]:
test_json[0]

{'input': 'According to the United States Census Bureau , the CDP covers an area of , 99.69 % of it land , and 0.31 % of it water . What is the attribute related to entity the CDP with quantity 99.69 %?',
 'output': 'area',
 'entity': 'the CDP',
 'quantity': '99.69 %',
 'sen': 'According to the United States Census Bureau , the CDP covers an area of , 99.69 % of it land , and 0.31 % of it water .'}

In [ ]:
res=[]
for i in range(len(test_json)):

  d=test_json[i]

  input=d['input']

  tokenized_inputs= tokenizer(input, max_length=256, truncation=True,return_tensors='pt')
  tokenized_inputs=tokenized_inputs.to('cuda')
  output_sequence = model.generate(**tokenized_inputs
                                 )

# decode the output sequence into text using the tokenizer
  #d=rule_test[i]
  u={}
  print(d['sen'])
  print("------------------------------Rule based result-------------------------------")
  print(f" Entity : {d['entity']}")
  print(f" Quantity : {d['quantity']} ")
  print(f" Attribute : {d['output']} ")
  print("-------------------------------- T5 Result------------------------------------")
  output_text = tokenizer.decode(output_sequence[0], skip_special_tokens=True)
  print(f" Attribute : {output_text}")
  print('\n\n')
  u['sentence']=d['sen']
  u['attribute']=output_text
  res.append(u)

In [ ]:
import json
with open ('/content/drive/MyDrive/MTP Project/out_attr/cv_300_attr/test_distribution_prediction.json','w') as f:
  json.dump(res,f)
